In [1]:
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('../output/trial_attentionwalk.csv')
with open('../idx2prot.pickle', 'rb') as f:
    idx2prot = pickle.load(f)

# df['protein'] = df['id'].apply(lambda x: idx2prot[x])
proteins = set(df['id'].to_list())
len(proteins)

7501

In [3]:
# Covert string to the proper set format
def get_set(x):                 
    x = x[2:-2]                 # Since each location is of the form {'location_name'}
    return set(x.split(','))

In [4]:
data = pd.read_csv('../input/trial.csv')

prot2loc = dict()
for i in data.index:

    l = get_set(data['locations'][i])

    p1 = data['protein1'][i]  
    try:
        prot2loc[p1] = prot2loc[p1].union(l)
    except:
        prot2loc[p1] = l

    p2 = data['protein2'][i]
    try:
        prot2loc[p2] = prot2loc[p2].union(l)
    except:
        prot2loc[p2] = l
p = next(iter(prot2loc))
print(p, prot2loc[p])
# list(prot2loc.values())[0]

0 {'Cell Junctions', 'Cytosol', 'Nucleoli fibrillar center', 'Nuclear membrane', 'Vesicles', 'Intermediate filaments', 'Golgi apparatus', 'Centrosome', 'Plasma membrane', 'Nuclear speckles', 'Endoplasmic reticulum', 'Mitochondria', 'Nucleoli', 'Microtubules', 'Nuclear bodies', 'Nucleoplasm'}


In [5]:
locs = ['Cytosol', 'Nucleoplasm', 'Plasma membrane', 'Vesicles',
        'Mitochondria', 'Endoplasmic reticulum', 'Golgi apparatus', 'Nucleoli',
        'Intermediate filaments', 'Centrosome', 'Nuclear speckles', 'Nuclear bodies',
        'Cell Junctions', 'Microtubules', 'Nuclear membrane', 'Nucleoli fibrillar center']
loc2id = {k: v for v, k in enumerate(locs)}

locs = list()
drop = list()
for i in df.index:
    loc = prot2loc[df['id'][i]]
    locs.append(loc)
    if len(loc) > 1:
        drop.append(i)
        
df['locations'] = locs
df = df.drop(drop)
df['locations'] = df['locations'].apply(lambda x: loc2id[next(iter(x))])
print(df.shape)
df.to_csv('../output/trial_data_attentionwalk.csv', index=None)

(6123, 130)


In [6]:
cols = list(df.columns)
cols.remove('locations')
cols.remove('id')
cols = ['locations']+cols

df = df[cols].sample(frac=1, random_state=42)

n_prots = df.shape[0]
alpha = 0.85
n_train = int(alpha * n_prots)
df_train = df.iloc[:n_train, :]
df_test = df.iloc[n_train:, :]

print(df_train.shape, df_test.shape)

(5204, 129) (919, 129)


In [7]:
df_train.to_csv('../output/train_attentionwalk.csv', index=None)
df_test.to_csv('../output/test_attentionwalk.csv', index=None)